In [ ]:
from sae_lens import SAE

sparse_autoencoder = SAE.from_pretrained(
    release="anhtu77/sae-tiny-stories-1L-21M",
    sae_id="sae",
    device="cuda",
)[0]

cfg.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

d:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tu7ph\.cache\huggingface\hub\models--anhtu77--sae-tiny-stories-1L-21M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sae_weights.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/65.6k [00:00<?, ?B/s]

d:\anaconda\Lib\site-packages\sae_lens\sae.py:146: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(
d:\anaconda\Lib\site-packages\sae_lens\sae.py:644: UserWarning: norm_scaling_factor not found for anhtu77/sae-tiny-stories-1L-21M and sae, but normalize_activations is 'expected_average_only_in'. Skipping normalization folding.
  warnings.warn(


In [7]:
from transformer_lens import HookedTransformer
import torch 
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "tiny-stories-1L-21M"
)  # This will wrap huggingface models and has lots of nice utilities.
# Print out the model architecture
print(model)

# Optionally, inspect the registered hooks in the model
print("\nRegistered hooks:")
print(list(model.hook_dict.keys()))

d:\anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_ou

In [18]:

prompt = "Once upon a time"
layer = 11

# Run the model with the prompt
tokens = model.to_tokens(prompt)
_, cache = model.run_with_cache(tokens)


In [19]:

# Get the activation of the specific layer
activation = cache["resid_post", layer]
logits = activation @ model.W_U
predicted_tokens = torch.argmax(logits, dim=-1)
decoded_tokens = model.to_str_tokens(predicted_tokens)
print(decoded_tokens)

['Story', ' upon', ' a', ' time', ' there']


In [20]:
import pandas as pd
import torch
# Let's start by getting the top 10 logits for each feature
projection_onto_unembed = sparse_autoencoder.W_dec @ model.W_U


# get the top 10 logits.
vals, inds = torch.topk(projection_onto_unembed, 10, dim=1)

# get 10 random features
random_indices = torch.randint(0, projection_onto_unembed.shape[0], (10,))

# Show the top 10 logits promoted by those features
top_10_logits_df = pd.DataFrame(
    [model.to_str_tokens(i) for i in inds[random_indices]],
    index=random_indices.tolist(),
).T
top_10_logits_df

NameError: name 'sparse_autoencoder' is not defined